In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'E:\DATASET MDVR IMG ALL\EXPERIMENT\Integeral\enhanced_integral_feature_data1.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (Random Forest)
param_dist = {
    'n_estimators': randint(50, 500),  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'bootstrap': [True, False]  # Whether to bootstrap samples
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (Random Forest)
    clf = RandomForestClassifier(random_state=42)
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (Random Forest)
        clf = RandomForestClassifier(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('feature_comparison_results_with_80_20_and_k_fold_RF.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.8922
5-Fold Accuracy: 0.8517 ± 0.0236
10-Fold Accuracy: 0.8477 ± 0.0419

Feature Set: Spectrogram
80:20 Accuracy: 0.7843
5-Fold Accuracy: 0.7652 ± 0.0166
10-Fold Accuracy: 0.7672 ± 0.0463

Feature Set: Spectral
80:20 Accuracy: 0.8088
5-Fold Accuracy: 0.7436 ± 0.0234
10-Fold Accuracy: 0.7475 ± 0.0260

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9990 ± 0.0020
10-Fold Accuracy: 0.9902 ± 0.0145

Feature Set: All_Features
80:20 Accuracy: 0.9951
5-Fold Accuracy: 0.9833 ± 0.0050
10-Fold Accuracy: 0.9853 ± 0.0140



In [2]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'feature_comparison_results_with_80_20_and_k_fold_RF.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data



Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.892157         0.914286        0.8000  0.853333   
1    Spectrogram        0.784314         0.736842        0.7000  0.717949   
2       Spectral        0.808824         0.836066        0.6375  0.723404   
3  Voice_Quality        1.000000         1.000000        1.0000  1.000000   
4   All_Features        0.995098         1.000000        0.9875  0.993711   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.956250   0.772945        0.382231           0.768660   0.107843   
1       0.842641   0.544000        0.485282           0.543531   0.215686   
2       0.874093   0.593834        0.448036           0.581351   0.191176   
3       1.000000   1.000000        0.000514           1.000000   0.000000   
4       0.999899   0.989747        0.133369           0.989695   0.004902   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.082627                 0.026475             0.083955   
1  ...            0.072026                 0.042982             0.102984   
2  ...            0.050938                 0.028227             0.058940   
3  ...            0.019711                 0.000000             0.029756   
4  ...            0.018960                 0.000751             0.028779   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  0.048182                     0.101375             0.041942   
1                  0.042099                     0.102809             0.046349   
2                  0.033177                     0.061778             0.025983   
3                  0.038839                     0.031156             0.014542   
4                  0.033728                     0.030040             0.014041   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.023122                      10.434770   
1                     0.047117                       2.166830   
2                     0.038654                       1.118422   
3                     0.000000                       0.696254   
4                     0.007432                       7.394470   

   10-Fold Std Dev testing_time  \
0                      0.014993   
1                      0.014757   
2                      0.012604   
3                      0.004835   
4                      0.014825   

                                   Best Params 80:20  
0  {'bootstrap': False, 'max_depth': 30, 'max_fea...  
1  {'bootstrap': True, 'max_depth': 50, 'max_feat...  
2  {'bootstrap': False, 'max_depth': None, 'max_f...  
3  {'bootstrap': True, 'max_depth': 30, 'max_feat...  
4  {'bootstrap': False, 'max_depth': 10, 'max_fea...  

[5 rows x 62 columns]

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'E:\DATASET MDVR IMG ALL\EXPERIMENT\Integeral\enhanced_integral_feature_data1.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (KNN)
param_dist = {
    'n_neighbors': randint(1, 50),  # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function used in prediction
    'p': [1, 2]  # Power parameter for Minkowski distance (1: Manhattan, 2: Euclidean)
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (KNN)
    clf = KNeighborsClassifier()
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (KNN)
        clf = KNeighborsClassifier()
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('feature_comparison_results_with_80_20_and_k_fold_KNN.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.8922
5-Fold Accuracy: 0.9047 ± 0.0209
10-Fold Accuracy: 0.9096 ± 0.0181

Feature Set: Spectrogram
80:20 Accuracy: 0.6618
5-Fold Accuracy: 0.6336 ± 0.0257
10-Fold Accuracy: 0.6365 ± 0.0351

Feature Set: Spectral
80:20 Accuracy: 0.6324
5-Fold Accuracy: 0.6464 ± 0.0369
10-Fold Accuracy: 0.6297 ± 0.0283

Feature Set: Voice_Quality
80:20 Accuracy: 0.9755
5-Fold Accuracy: 0.9135 ± 0.0588
10-Fold Accuracy: 0.9509 ± 0.0384

Feature Set: All_Features
80:20 Accuracy: 0.7794
5-Fold Accuracy: 0.6671 ± 0.0389
10-Fold Accuracy: 0.6797 ± 0.0499



In [4]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'feature_comparison_results_with_80_20_and_k_fold_KNN.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.892157         0.902778        0.8125  0.855263   
1    Spectrogram        0.661765         0.677419        0.2625  0.378378   
2       Spectral        0.632353         0.533333        0.5000  0.516129   
3  Voice_Quality        0.975490         0.987013        0.9500  0.968153   
4   All_Features        0.779412         0.777778        0.6125  0.685315   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.952016   0.772418        0.565337           0.769704   0.107843   
1       0.681149   0.247330        0.633610           0.204026   0.338235   
2       0.657258   0.220482        7.298649           0.220183   0.367647   
3       0.997581   0.948702        0.091622           0.948244   0.024510   
4       0.821270   0.527953        2.255825           0.519170   0.220588   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.025223                 0.017780             0.038316   
1  ...            0.097528                 0.072176             0.102578   
2  ...            0.072122                 0.032109             0.073380   
3  ...            0.050827                 0.012124             0.081442   
4  ...            0.101916                 0.050066             0.117578   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  0.581075                     0.039054             0.018122   
1                  0.703937                     0.087523             0.035110   
2                  1.832646                     0.073296             0.028321   
3                  0.044945                     0.081677             0.038449   
4                  2.045808                     0.121135             0.049920   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.021990                       0.123985   
1                     0.052091                       0.007359   
2                     0.036701                       0.018304   
3                     0.027047                       0.018629   
4                     0.045511                       0.028493   

   10-Fold Std Dev testing_time  \
0                      0.001300   
1                      0.000988   
2                      0.001748   
3                      0.001653   
4                      0.003094   

                                   Best Params 80:20  
0   {'n_neighbors': 7, 'p': 1, 'weights': 'uniform'}  
1  {'n_neighbors': 14, 'p': 1, 'weights': 'distan...  
2  {'n_neighbors': 2, 'p': 2, 'weights': 'distance'}  
3  {'n_neighbors': 21, 'p': 1, 'weights': 'distan...  
4   {'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}  

[5 rows x 62 columns]

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'E:\DATASET MDVR IMG ALL\EXPERIMENT\Integeral\enhanced_integral_feature_data1.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (Decision Tree)
param_dist = {
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'criterion': ['gini', 'entropy']  # Function to measure the quality of a split
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (Decision Tree)
    clf = DecisionTreeClassifier(random_state=42)
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (Decision Tree)
        clf = DecisionTreeClassifier(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('feature_comparison_results_with_80_20_and_k_fold_DecisionTree.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.6961
5-Fold Accuracy: 0.7034 ± 0.0508
10-Fold Accuracy: 0.7327 ± 0.0353

Feature Set: Spectrogram
80:20 Accuracy: 0.6863
5-Fold Accuracy: 0.7092 ± 0.0231
10-Fold Accuracy: 0.6866 ± 0.0578

Feature Set: Spectral
80:20 Accuracy: 0.7500
5-Fold Accuracy: 0.7230 ± 0.0170
10-Fold Accuracy: 0.7142 ± 0.0297

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9470 ± 0.0414
10-Fold Accuracy: 0.9441 ± 0.0509

Feature Set: All_Features
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.8280 ± 0.0883
10-Fold Accuracy: 0.8664 ± 0.0829



In [6]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'feature_comparison_results_with_80_20_and_k_fold_DecisionTree.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.696078         0.604651        0.6500  0.626506   
1    Spectrogram        0.686275         0.648148        0.4375  0.522388   
2       Spectral        0.750000         0.698630        0.6375  0.666667   
3  Voice_Quality        1.000000         1.000000        1.0000  1.000000   
4   All_Features        1.000000         1.000000        1.0000  1.000000   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.724395   0.371561    8.582311e+00           0.370871   0.303922   
1       0.713609   0.314594    1.430113e+00           0.301669   0.313725   
2       0.754637   0.468590    3.196866e+00           0.467336   0.250000   
3       1.000000   1.000000    2.220446e-16           1.000000   0.000000   
4       1.000000   1.000000    2.220446e-16           1.000000   0.000000   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.042563                 0.029073             0.072198   
1  ...            0.072115                 0.069750             0.123931   
2  ...            0.048436                 0.026998             0.065931   
3  ...            0.062762                 0.018753             0.103114   
4  ...            0.107663                 0.055355             0.172027   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  1.360980                     0.070970             0.035313   
1                  1.445893                     0.119689             0.057841   
2                  1.837755                     0.066958             0.029674   
3                  0.293789                     0.105312             0.050946   
4                  0.964964                     0.174294             0.082884   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.058262                       0.093603   
1                     0.068972                       0.020012   
2                     0.027890                       0.024467   
3                     0.055767                       0.015156   
4                     0.075434                       0.080443   

   10-Fold Std Dev testing_time  \
0                      0.000400   
1                      0.000607   
2                      0.000782   
3                      0.001007   
4                      0.000448   

                                   Best Params 80:20  
0  {'criterion': 'entropy', 'max_depth': 100, 'ma...  
1  {'criterion': 'gini', 'max_depth': 100, 'max_f...  
2  {'criterion': 'entropy', 'max_depth': 50, 'max...  
3  {'criterion': 'gini', 'max_depth': 50, 'max_fe...  
4  {'criterion': 'entropy', 'max_depth': 50, 'max...  

[5 rows x 62 columns]

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'E:\DATASET MDVR IMG ALL\EXPERIMENT\Integeral\enhanced_integral_feature_data1.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (AdaBoost)
param_dist = {
    'n_estimators': randint(50, 500),  # Number of boosting stages
    'learning_rate': [0.01, 0.1, 0.5, 1.0],  # Learning rate
    'algorithm': ['SAMME', 'SAMME.R']  # Algorithm to use
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Initialize the classifier with RandomizedSearchCV (AdaBoost)
    clf = AdaBoostClassifier(random_state=42)
    random_search = RandomizedSearchCV(
        estimator=clf, param_distributions=param_dist, n_iter=50,
        scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    random_search.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    best_clf = random_search.best_estimator_
    
    # Test the model
    start_test_time = time.time()
    y_pred = best_clf.predict(X_test)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics, random_search.best_params_

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Initialize the classifier with RandomizedSearchCV (AdaBoost)
        clf = AdaBoostClassifier(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=clf, param_distributions=param_dist, n_iter=50,
            scoring='accuracy', cv=5, verbose=0, random_state=42, n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        random_search.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        best_clf = random_search.best_estimator_
        
        # Test the model
        start_test_time = time.time()
        y_pred = best_clf.predict(X_val)
        y_pred_proba = best_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20, best_params_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()},
        'Best Params 80:20': best_params_80_20
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('feature_comparison_results_with_80_20_and_k_fold_AdaBoost.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.7843
5-Fold Accuracy: 0.7976 ± 0.0278
10-Fold Accuracy: 0.7858 ± 0.0574

Feature Set: Spectrogram
80:20 Accuracy: 0.7059
5-Fold Accuracy: 0.6847 ± 0.0196
10-Fold Accuracy: 0.6630 ± 0.0359

Feature Set: Spectral
80:20 Accuracy: 0.7402
5-Fold Accuracy: 0.6994 ± 0.0161
10-Fold Accuracy: 0.6915 ± 0.0339

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9735 ± 0.0147
10-Fold Accuracy: 0.9705 ± 0.0252

Feature Set: All_Features
80:20 Accuracy: 1.0000
5-Fold Accuracy: 0.9843 ± 0.0105
10-Fold Accuracy: 0.9666 ± 0.0278



In [9]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'feature_comparison_results_with_80_20_and_k_fold_AdaBoost.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.784314         0.743243        0.6875  0.714286   
1    Spectrogram        0.705882         0.651515        0.5375  0.589041   
2       Spectral        0.740196         0.736842        0.5250  0.613139   
3  Voice_Quality        1.000000         1.000000        1.0000  1.000000   
4   All_Features        1.000000         1.000000        1.0000  1.000000   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.854435   0.542541        0.684320           0.541479   0.215686   
1       0.734980   0.367373        0.632386           0.363296   0.294118   
2       0.772530   0.439618        0.681319           0.425749   0.259804   
3       1.000000   1.000000        0.549805           1.000000   0.000000   
4       1.000000   1.000000        0.551669           1.000000   0.000000   

   ...  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  10-Fold Std Dev mcc  \
0  ...            0.073899                 0.051274             0.121224   
1  ...            0.166816                 0.039562             0.095099   
2  ...            0.095108                 0.032744             0.085773   
3  ...            0.036031                 0.015200             0.050635   
4  ...            0.037285                 0.012176             0.058149   

   10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  10-Fold Std Dev mae  \
0                  0.023585                     0.120273             0.057388   
1                  0.016053                     0.112481             0.035853   
2                  0.023537                     0.089836             0.033866   
3                  0.016264                     0.054935             0.025179   
4                  0.027523                     0.059398             0.027845   

   10-Fold Std Dev specificity  10-Fold Std Dev training_time  \
0                     0.059793                      12.858045   
1                     0.090777                       0.472609   
2                     0.029960                       7.543750   
3                     0.000000                       0.628795   
4                     0.016209                       1.327813   

   10-Fold Std Dev testing_time  \
0                      0.031554   
1                      0.026194   
2                      0.030885   
3                      0.026366   
4                      0.018771   

                                   Best Params 80:20  
0  {'algorithm': 'SAMME.R', 'learning_rate': 0.5,...  
1  {'algorithm': 'SAMME', 'learning_rate': 1.0, '...  
2  {'algorithm': 'SAMME.R', 'learning_rate': 0.1,...  
3  {'algorithm': 'SAMME', 'learning_rate': 1.0, '...  
4  {'algorithm': 'SAMME', 'learning_rate': 1.0, '...  

[5 rows x 62 columns]

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, log_loss, cohen_kappa_score, mean_absolute_error, confusion_matrix
from scipy.stats import randint
import time

# Load the CSV file with extracted features
csv_output_path = r'E:\DATASET MDVR IMG ALL\EXPERIMENT\Integeral\enhanced_integral_feature_data1.csv'
data = pd.read_csv(csv_output_path)

# Remove any rows with null values
data.dropna(inplace=True)

# Define feature groups based on feature types
feature_groups = {
    'MFCC': ['mfcc_1', 'mfcc_delta_1', 'mfcc_delta2_1', 'mfcc_2', 'mfcc_delta_2', 'mfcc_delta2_2', 
             'mfcc_3', 'mfcc_delta_3', 'mfcc_delta2_3', 'mfcc_4', 'mfcc_delta_4', 'mfcc_delta2_4', 
             'mfcc_5', 'mfcc_delta_5', 'mfcc_delta2_5', 'mfcc_6', 'mfcc_delta_6', 'mfcc_delta2_6', 
             'mfcc_7', 'mfcc_delta_7', 'mfcc_delta2_7', 'mfcc_8', 'mfcc_delta_8', 'mfcc_delta2_8', 
             'mfcc_9', 'mfcc_delta_9', 'mfcc_delta2_9', 'mfcc_10', 'mfcc_delta_10', 'mfcc_delta2_10', 
             'mfcc_11', 'mfcc_delta_11', 'mfcc_delta2_11', 'mfcc_12', 'mfcc_delta_12', 'mfcc_delta2_12', 
             'mfcc_13', 'mfcc_delta_13', 'mfcc_delta2_13', 'entropy', 'kurtosis', 'skewness'],
    'Spectrogram': ['mel_spectrogram', 'chroma_stft', 'entropy', 'kurtosis', 'skewness'],
    'Spectral': ['rmse', 'spectral_contrast', 'spectral_flatness', 'spectral_centroid', 
                 'spectral_bandwidth', 'spectral_rolloff', 'zcr', 'entropy', 'kurtosis', 'skewness'],
    'Voice_Quality': ['jitter_local', 'shimmer_local', 'f0_mean', 'hnr', 'entropy', 'kurtosis', 'skewness']
}

# Add a new feature group that combines all features
feature_groups['All_Features'] = list(set().union(*[features for features in feature_groups.values()]))

# Parameter grid for RandomizedSearchCV (Random Forest)
param_dist_rf = {
    'n_estimators': randint(50, 500),  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'bootstrap': [True, False]  # Whether to bootstrap samples
}

# Parameter grid for RandomizedSearchCV (Decision Tree)
param_dist_dt = {
    'max_depth': [None, 10, 20, 30, 50, 100],  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'criterion': ['gini', 'entropy']  # Function to measure the quality of a split
}

# Parameter grid for RandomizedSearchCV (AdaBoost)
param_dist_ab = {
    'n_estimators': randint(50, 500),  # Number of boosting stages
    'learning_rate': [0.01, 0.1, 0.5, 1.0],  # Learning rate
    'algorithm': ['SAMME', 'SAMME.R']  # Algorithm to use
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'mcc': matthews_corrcoef(y_true, y_pred),
        'log_loss': log_loss(y_true, y_pred_proba),
        'cohen_kappa': cohen_kappa_score(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'specificity': confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1])
    }
    return metrics

# Function to evaluate model using 80:20 train-test split
def evaluate_80_20_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Define base models
    rf = RandomForestClassifier(random_state=42)
    dt = DecisionTreeClassifier(random_state=42)
    ab = AdaBoostClassifier(random_state=42)
    
    # Define meta-classifier
    meta_classifier = LogisticRegression(random_state=42)
    
    # Define stacking classifier
    stacking_clf = StackingClassifier(
        estimators=[('rf', rf), ('dt', dt), ('ab', ab)],
        final_estimator=meta_classifier,
        stack_method='auto',
        n_jobs=-1
    )
    
    # Train the model
    start_train_time = time.time()
    stacking_clf.fit(X_train, y_train)
    end_train_time = time.time()
    training_time = end_train_time - start_train_time
    
    # Test the model
    start_test_time = time.time()
    y_pred = stacking_clf.predict(X_test)
    y_pred_proba = stacking_clf.predict_proba(X_test)[:, 1]
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time
    
    # Calculate performance metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    metrics['training_time'] = training_time
    metrics['testing_time'] = testing_time
    
    return metrics

# Function to evaluate model using k-fold cross-validation
def evaluate_k_fold_cv(X, y, k):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': [],
        'mcc': [],
        'log_loss': [],
        'cohen_kappa': [],
        'mae': [],
        'specificity': [],
        'training_time': [],
        'testing_time': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Define base models
        rf = RandomForestClassifier(random_state=42)
        dt = DecisionTreeClassifier(random_state=42)
        ab = AdaBoostClassifier(random_state=42)
        
        # Define meta-classifier
        meta_classifier = LogisticRegression(random_state=42)
        
        # Define stacking classifier
        stacking_clf = StackingClassifier(
            estimators=[('rf', rf), ('dt', dt), ('ab', ab)],
            final_estimator=meta_classifier,
            stack_method='auto',
            n_jobs=-1
        )
        
        # Train the model
        start_train_time = time.time()
        stacking_clf.fit(X_train, y_train)
        end_train_time = time.time()
        training_time = end_train_time - start_train_time
        
        # Test the model
        start_test_time = time.time()
        y_pred = stacking_clf.predict(X_val)
        y_pred_proba = stacking_clf.predict_proba(X_val)[:, 1]
        end_test_time = time.time()
        testing_time = end_test_time - start_test_time
        
        # Calculate performance metrics
        metrics = calculate_metrics(y_val, y_pred, y_pred_proba)
        for key in metrics:
            fold_metrics[key].append(metrics[key])
        fold_metrics['training_time'].append(training_time)
        fold_metrics['testing_time'].append(testing_time)
    
    # Calculate mean and std dev of performance metrics
    mean_metrics = {k: np.mean(v) for k, v in fold_metrics.items()}
    std_metrics = {k: np.std(v) for k, v in fold_metrics.items()}
    
    return mean_metrics, std_metrics

# Store results for comparison
results = []

# Loop through each feature group
for group_name, features in feature_groups.items():
    X = data[features]
    y = data['label']
    
    # Evaluate using 80:20 train-test split
    metrics_80_20 = evaluate_80_20_split(X, y)
    
    # Evaluate using 5-fold cross-validation
    mean_metrics_5_fold, std_metrics_5_fold = evaluate_k_fold_cv(X, y, 5)
    
    # Evaluate using 10-fold cross-validation
    mean_metrics_10_fold, std_metrics_10_fold = evaluate_k_fold_cv(X, y, 10)
    
    # Store results for this feature group
    results.append({
        'Feature Set': group_name,
        **{f'80:20 {k}': v for k, v in metrics_80_20.items()},
        **{f'5-Fold Mean {k}': v for k, v in mean_metrics_5_fold.items()},
        **{f'5-Fold Std Dev {k}': v for k, v in std_metrics_5_fold.items()},
        **{f'10-Fold Mean {k}': v for k, v in mean_metrics_10_fold.items()},
        **{f'10-Fold Std Dev {k}': v for k, v in std_metrics_10_fold.items()}
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Print accuracy with ± symbol for standard deviation
for index, row in results_df.iterrows():
    print(f"Feature Set: {row['Feature Set']}")
    print(f"80:20 Accuracy: {row['80:20 accuracy']:.4f}")
    print(f"5-Fold Accuracy: {row['5-Fold Mean accuracy']:.4f} ± {row['5-Fold Std Dev accuracy']:.4f}")
    print(f"10-Fold Accuracy: {row['10-Fold Mean accuracy']:.4f} ± {row['10-Fold Std Dev accuracy']:.4f}")
    print()

# Optionally, save the results to a CSV file
results_df.to_csv('feature_comparison_results_with_80_20_and_k_fold_Stacking.csv', index=False)

Feature Set: MFCC
80:20 Accuracy: 0.8480
5-Fold Accuracy: 0.8016 ± 0.0456
10-Fold Accuracy: 0.8084 ± 0.0357

Feature Set: Spectrogram
80:20 Accuracy: 0.7843
5-Fold Accuracy: 0.7583 ± 0.0276
10-Fold Accuracy: 0.7584 ± 0.0288

Feature Set: Spectral
80:20 Accuracy: 0.8333
5-Fold Accuracy: 0.7505 ± 0.0159
10-Fold Accuracy: 0.7663 ± 0.0312

Feature Set: Voice_Quality
80:20 Accuracy: 1.0000
5-Fold Accuracy: 1.0000 ± 0.0000
10-Fold Accuracy: 1.0000 ± 0.0000

Feature Set: All_Features
80:20 Accuracy: 0.9902
5-Fold Accuracy: 0.9636 ± 0.0201
10-Fold Accuracy: 0.9647 ± 0.0165



In [11]:
import pandas as pd

# Define the path to the CSV file
csv_output_path = r'feature_comparison_results_with_80_20_and_k_fold_Stacking.csv'

# Load the data
data = pd.read_csv(csv_output_path)

data


Feature Set  80:20 accuracy  80:20 precision  80:20 recall  80:20 f1  \
0           MFCC        0.848039         0.855072        0.7375  0.791946   
1    Spectrogram        0.784314         0.743243        0.6875  0.714286   
2       Spectral        0.833333         0.838235        0.7125  0.770270   
3  Voice_Quality        1.000000         1.000000        1.0000  1.000000   
4   All_Features        0.990196         0.975610        1.0000  0.987654   

   80:20 roc_auc  80:20 mcc  80:20 log_loss  80:20 cohen_kappa  80:20 mae  \
0       0.929536   0.677850        0.350387           0.673280   0.151961   
1       0.830544   0.542541        0.487429           0.541479   0.215686   
2       0.880141   0.646057        0.435600           0.640845   0.166667   
3       1.000000   1.000000        0.010849           1.000000   0.000000   
4       0.999798   0.979732        0.040713           0.979526   0.009804   

   ...  10-Fold Std Dev recall  10-Fold Std Dev f1  10-Fold Std Dev roc_auc  \
0  ...                0.085725            0.073216             2.966379e-02   
1  ...                0.078369            0.058789             4.290098e-02   
2  ...                0.089942            0.072003             2.506054e-02   
3  ...                0.000000            0.000000             6.080942e-17   
4  ...                0.043552            0.023380             1.511899e-03   

   10-Fold Std Dev mcc  10-Fold Std Dev log_loss  10-Fold Std Dev cohen_kappa  \
0             0.076417                  0.019007                     0.086318   
1             0.066557                  0.017428                     0.068587   
2             0.067645                  0.012563                     0.079478   
3             0.000000                  0.015026                     0.000000   
4             0.033178                  0.033101                     0.035894   

   10-Fold Std Dev mae  10-Fold Std Dev specificity  \
0             0.035658                     0.029784   
1             0.028764                     0.036935   
2             0.031169                     0.020134   
3             0.000000                     0.000000   
4             0.016493                     0.004839   

   10-Fold Std Dev training_time  10-Fold Std Dev testing_time  
0                       0.112219                      0.006151  
1                       0.044727                      0.003415  
2                       0.095348                      0.004326  
3                       0.012119                      0.001124  
4                       0.107924                      0.002855  

[5 rows x 61 columns]